In [ ]:
from ultralytics import YOLO
import cv2, numpy as np

# ——————————————————————————————————————————
# STEP 1: Your existing offside logic, refactored to a function:
def run_offside_on(frame, line_segs, players):
    """
    line_segs: { cls_name: [(x1,y1,x2,y2), ...], ... }
    player_feet: { cls_name: [(fx,fy), ...], ... }
    Draws lines, picks nearest defender/attacker foot, computes vanishing point,
    draws offside lines and text. Exactly your existing code, but replacing
    JSON reads with these parameters.
    """
    h, w = frame.shape[:2]

    # 2) pick touch_cls & horizontal_cls from line_segs.keys()
    line_classes = set(line_segs.keys())
    #print(line_classes)
    
    # priority: Side line → Big rect. → Small rect.
    if "side_right" in line_classes:
        touch_cls = "side_right"
    elif "side_left" in line_classes:
        touch_cls = "side_left"
    elif "rect_big_right_main" in line_classes:
        touch_cls = "rect_big_right_main"
    elif "rect_big_left_main" in line_classes:
        touch_cls = "rect_big_left_main"
    elif "rect_small_right_main" in line_classes:
        touch_cls = "rect_small_right_main"
    elif "rect_small_left_main" in line_classes:
        touch_cls = "rect_small_left_main"
    else:
        raise ValueError("No usable vertical field‐edge found in annotations")

    # always use Side line top as the horizontal reference
    if   "side_top"    in line_classes: horizontal_cls = "side_top"
    elif "side_bottom" in line_classes: horizontal_cls = "side_bottom"
    elif "rect_big_right_top" in line_classes: horizontal_cls = "rect_big_right_top"
    elif "rect_big_right_bottom" in line_classes: horizontal_cls = "rect_big_right_bottom"
    elif "rect_big_left_top" in line_classes: horizontal_cls = "rect_big_left_top"
    elif "rect_big_left_bottom" in line_classes: horizontal_cls = "rect_big_left_bottom"
    else:
        raise RuntimeError("No horizontal pitch edge found in annotations")
    
    print(touch_cls)
    print(horizontal_cls)

    # 3) extract endpoints(A_top,B_top) & (A_side,B_side) from line_segs[touch_cls] etc.
    def endpoints_from_box(box, kind):
        (x1, y1), (x2, y2) = box
        if kind == 'horizontal':
            # bottom‐left → top‐right
            pt1 = np.array([x1, y2], dtype=float)
            pt2 = np.array([x2, y1], dtype=float)
        elif kind == 'vertical':
            # top‐left → bottom‐right
            pt1 = np.array([x1, y1], dtype=float)
            pt2 = np.array([x2, y2], dtype=float)
        else:
            raise ValueError("kind must be 'horizontal' or 'vertical'")
        return pt1, pt2
    #print(line_segs)
    h_box      = line_segs[horizontal_cls]
    #print(h_box)
    v_box      = line_segs[touch_cls]
    #print(v_box)

    A_top,  B_top  = endpoints_from_box(h_box, 'horizontal')
    #print(A_top, B_top)
    A_side, B_side = endpoints_from_box(v_box, 'vertical')
    #print(A_side, B_side)

    # … then copy & paste the rest of your existing code from “# draw references”
    # through “# Show result”, replacing every `ann["lines"]` with `line_segs[...]`
    # and every `ann["bboxes"]` foot‐loop with `player_feet[...]`.

    # (For brevity I’m not re-pasting the 200 lines you already have;
    #  just drop them in here verbatim, swapping data sources.)

    # draw references
    cv2.line(frame, tuple(A_top.astype(int)),  tuple(B_top.astype(int)),  (255,255,255),2)
    cv2.line(frame, tuple(A_side.astype(int)), tuple(B_side.astype(int)), (255,255,255),2)

    # --- 3) Extend touch-line to bottom or top so it spans full frame ---
    d_line = B_side - A_side
    if abs(d_line[1])>1e-6:
        # extend downward if right touch-line, upward if left
        target_y = h if touch_cls.endswith("right") else 0
        t = (target_y - A_side[1]) / d_line[1]
        B_ext = A_side + d_line * t
    else:
        B_ext = B_side.copy()
    cv2.line(frame, tuple(A_side.astype(int)), tuple(B_ext.astype(int)), (200,200,255),2)

    # --- 5) Pick defender & attacker by perp-distance to touch-line along u_top ---
    def pick_closest(team_cls, color):
        # 1) original side‐line vector & its length
        side_vec   = B_side - A_side
        side_len   = np.linalg.norm(side_vec)

        print(side_len)
        print(h)

        # 2) unit‐vector along top‐line for fallback
        u_top = (B_top - A_top)
        u_top = (u_top / np.linalg.norm(u_top)).astype(np.float32)

        # 3) a threshold on the raw annotation length
        min_len = 0.5 * h   # e.g. 50% of frame height

        best = None
        for pl in players:
            name,_,(x1,y1,x2,y2) = pl[0]
            if name != team_cls:
                continue

            # choose correct foot‐x
            if touch_cls.endswith("right"):
                fx = x2
            else:
                fx = x1
            fy = y2
            P  = np.array([fx, fy], dtype=float)

            # 4) if the annotated side‐line is long enough, use the true perp‐distance
            if side_len > min_len:
                dist = abs(np.cross(side_vec, P - A_side)) / side_len
            else:
                # fallback: project along the goal‐line direction
                dist = abs((P - A_side).dot(u_top))

            if best is None or dist < best[0]:
                best = (dist, P, (x1,y1,x2,y2))
        if best == None:
            raise RuntimeError("Players not detected for one or teams")
        # 5) draw the chosen box & foot
        d, foot, (x1,y1,x2,y2) = best
        cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),color,2)
        cv2.circle(   frame,(int(foot[0]),int(foot[1])),6,color,-1)

        return best

    # defender = same-side team as touch-line (if right touch-line, defender team right)
    def_team = "player_right" if touch_cls=="side_right" else "player_left"
    att_team = "player_left"  if touch_cls=="side_right" else "player_right"
    dist_def, def_foot, _ = pick_closest(def_team,(0,255,0))
    dist_att, att_foot, _ = pick_closest(att_team,(0,255,255))

    # --- 6) Compute vanishing point V for “vertical” references ---
    vert_classes = {touch_cls, 
                    "rect_big_right_main" if touch_cls.endswith("right") else "rect_big_left_main",
                    "rect_small_right_main" if touch_cls.endswith("right") else "rect_small_left_main"}
    lines = []
    print(line_segs.items())
    for cl, ln in line_segs.items():
        if cl in vert_classes:
            pts = np.array(ln,float).reshape(-1,2)
            A,B = pts[pts[:,1].argmin()], pts[pts[:,1].argmax()]
            lines.append(np.cross(np.append(A,1), np.append(B,1)))
    L = np.stack(lines,axis=0)
    _,_,vh = np.linalg.svd(L)
    V = vh[-1]; V/=V[2]
    vx, vy = V[0], V[1]

    # --- 7) Draw offside lines (parallel to touch-line) through each foot ---
    Lspan = max(h,w)*1.5
    for foot, col in [(def_foot,(0,0,255)), (att_foot,(255,0,255))]:
        dir_img = np.array([vx,vy]) - foot
        dir_img /= np.linalg.norm(dir_img)
        P1 = foot - dir_img*Lspan
        P2 = foot + dir_img*Lspan
        cv2.line(frame, tuple(P1.astype(int)), tuple(P2.astype(int)), col,3)

    # --- 8) OFFSIDE decision ---
    if dist_att < dist_def:
        cv2.putText(frame,"OFFSIDE",(50,80),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

    return frame

# ——————————————————————————————————————————
# STEP 2: Wrapper that runs YOLO and builds those dicts:

def detect_and_offside(img_path, yolo_weights):
    # 1) read
    frame = cv2.imread(img_path)
    h, w = frame.shape[:2]

    # 2) run YOLO
    model   = YOLO(yolo_weights)
    results = model(frame, conf=0.25, imgsz=640)[0]

    # 3) extract line segments & players
    # line-like classes map
    horizontal = {
        'side_top','side_bottom',
        'rect_big_left_top','rect_big_right_top',
        'rect_big_left_bottom','rect_big_right_bottom'
    }
    vertical   = {
        'side_left','side_right',
        'rect_big_left_main','rect_big_right_main',
        'rect_small_left_main','rect_small_right_main'
    }

    right_classes = {
        'side_right', 
        'rect_big_right_main',
        'rect_small_right_main'
        }
    left_classes = {
        'side_left', 
        'rect_big_left_main',
        'rect_small_left_main'
        }

    line_boxes = {}
    raw_players = []
    for box in results.boxes:
        x1,y1,x2,y2 = box.xyxy[0].cpu().numpy().astype(int)
        cls_id      = int(box.cls[0].cpu().numpy())
        name        = model.names[cls_id]
        conf        = float(box.conf[0].cpu().numpy())

        if name in horizontal or name in vertical:
            # keep only the most confident segment per class
            prev = line_boxes.get(name)
            if prev is None or conf>prev[1]:
                line_boxes[name] = ((x1,y1,x2,y2), conf)
        else:
            # gather all player/keeper boxes for NMS‐across‐classes
            raw_players.append((name, conf, (x1,y1,x2,y2)))

    # unpack line_segs to just geometry
    #line_boxes = {k:[v[0]] for k,v in line_boxes.items()}
    #print(line_boxes)

    flag=False
    #print(lines)
    for name, (box, conf) in line_boxes.items():
        if name in right_classes:
            flag=True
            break
        elif name in left_classes:
            flag=False
            break
    print(flag)    

    line_segs = {}
    #print(line_boxes.items())
    for name, (box, conf) in line_boxes.items():
        #print(box)
        (x1, y1, x2, y2) = box
        if flag:
            if name in horizontal:
                # bottom‐left → top‐right
                pt1 = (x1, y1)
                pt2 = (x2, y2)
            elif name in vertical:
                # top‐left → bottom‐right
                pt1 = (x1, y1)
                pt2 = (x2, y2)
            else:
                # fallback: draw its box
                pt1 = (x1, y1)
                pt2 = (x2, y2)
        else:
            if name in horizontal:
                # top‐left → bottom‐right
                pt1 = (x1, y2)
                pt2 = (x2, y1)
            elif name in vertical:
                # bottom‐left → top‐right
                pt1 = (x1, y2)
                pt2 = (x2, y1)
            else:
                # fallback: draw its box
                pt1 = (x1, y1)
                pt2 = (x2, y2)

        # store the one true segment for this class
        line_segs[name] = (pt1, pt2)

    # 4) do simple across‐class NMS for players (your clustering code)
    #    to get final list of (name, (fx,fy)) foot points:
    clusters = []
    def iou(a,b):
        x1,y1,x2,y2 = a; X1,Y1,X2,Y2=b
        xi1, yi1 = max(x1,X1), max(y1,Y1)
        xi2, yi2 = min(x2,X2), min(y2,Y2)
        if xi2<=xi1 or yi2<=yi1: return 0.0
        inter = (xi2-xi1)*(yi2-yi1)
        union = (x2-x1)*(y2-y1)+(X2-X1)*(Y2-Y1)-inter
        return inter/union

    for name,conf,box in sorted(raw_players, key=lambda x:-x[1]):
        placed=False
        for cl in clusters:
            if any(iou(box,m[2])>0.5 for m in cl):
                cl.append((name,conf,box))
                placed=True
                break
        if not placed:
            clusters.append([(name,conf,box)])
    #print(clusters)
    players = clusters

    # now from each cluster take the top one and compute foot point
    # player_feet = {}
    # for cl in clusters:
    #     name,conf,(x1,y1,x2,y2) = cl[0]
    #     # choose left/right foot
    #     fx = x2 if name.endswith('right') else x1
    #     fy = y2
    #     player_feet.setdefault(name, []).append((fx,fy))
    

    # 5) call your offside routine
    out = run_offside_on(frame, line_segs, players)

    # 6) show
    cv2.namedWindow("Offside", cv2.WINDOW_NORMAL)
    cv2.imshow("Offside", out)
    cv2.resizeWindow("Offside",960,int(960*h/w))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# ——————————————————————————————————————————
if __name__ == "__main__":
    detect_and_offside("4.png", "runs/train/offsidev32/weights/best.pt")